This notebook will read in all the cleaned tweet .csv files provided and write them to postgres tables, (1 per file)

it assumes you have unzipped the ProcessedTweetDataFeb file in this directory

In [1]:
import os
import numpy as np
import pandas as pd
import sqlalchemy as sq
import io
import re
pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)

In [2]:
PG_CONN_STRING = '***'
engine = sq.create_engine(PG_CONN_STRING)

/Users/ringhilterra/virtualenvs/py3dev/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [8]:
def write_results_pg(result_df, tweet_table):
    schema = 'ryan_test'
    result_df.head(0).to_sql(tweet_table, con=engine, schema=schema, index=False, if_exists='replace') #truncates the table
    conn = engine.raw_connection()
    cur = conn.cursor()
    cur.execute("SET search_path TO '{schema}'".format(schema=schema))
    output = io.StringIO()
    result_df.to_csv(output, sep='\t', header=False, index=False)
    output.seek(0)
    contents = output.getvalue()
    cur.copy_from(output, table=tweet_table, null="") # null values become ''
    conn.commit()
    print('completed write to table: {0}'.format(tweet_table))

In [9]:
# iterate through the tweet file directory and get all the file paths
file_list = []
day_dirs = os.listdir('ProcessedTweetDataFeb/')
for day_dir in day_dirs:
    #print(day_dir)
    if day_dir[0] != '.':
        fnames = os.listdir('ProcessedTweetDataFeb/'+day_dir)
        fnames = [('ProcessedTweetDataFeb/' + day_dir + '/' + x) for x in fnames]
        file_list = file_list + fnames

In [10]:
%%time
# read in each tweet file and do basic cleanup and write to postgres table
for afile in file_list:
    tweet_df = pd.read_csv(afile, lineterminator='\n')
    tweet_df = tweet_df[['id', 'tokens']]
    tweet_df['tokens'] = tweet_df['tokens'].astype('str') 
    tweet_df['tokens'] = tweet_df['tokens'].apply(lambda x: re.sub('\s+',' ',x) )
    tweet_table = afile.split('/')[-1].lower()
    tweet_table = tweet_table.replace('-', '_')
    tweet_table = tweet_table[:-4]
    write_results_pg(tweet_df, tweet_table)

completed write to table: tweetsprocessed2018_02_01_1
completed write to table: tweetsprocessed2018_02_01_2
completed write to table: tweetsprocessed2018_02_01_3
completed write to table: tweetsprocessed2018_02_01_4
completed write to table: tweetsprocessed2018_02_01_5
completed write to table: tweetsprocessed2018_02_01_6
completed write to table: tweetsprocessed2018_02_01_7
completed write to table: tweetsprocessed2018_02_10_1
completed write to table: tweetsprocessed2018_02_10_2
completed write to table: tweetsprocessed2018_02_10_3
completed write to table: tweetsprocessed2018_02_10_4
completed write to table: tweetsprocessed2018_02_10_5
completed write to table: tweetsprocessed2018_02_10_6
completed write to table: tweetsprocessed2018_02_10_7
completed write to table: tweetsprocessed2018_02_10_8
completed write to table: tweetsprocessed2018_02_02_1
completed write to table: tweetsprocessed2018_02_02_10
completed write to table: tweetsprocessed2018_02_02_11
completed write to table: 